In [1]:
import warnings

warnings.filterwarnings("ignore")

from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import re

In [2]:
df_mining = pd.read_csv('preprocessing_ml/polarity_readibility_features.csv', index_col=False)

In [3]:
df_mining.head()

,Coffee,Description,scores,valid_date,polarity_textblob,subjectivity_textblob,neg_nltk,neu_nltk,pos_nltk,compound_nltk,...,coleman_liau_index,ari,dale_chall_readability,difficult_words,text_standard,flesch_corrected,kincaid_grade_corrected,ari_corrected,dale_chall_readability_corrected,len_review
0,Réveille Coffee,"This is a cute coffee shop, I love the ambianc...",4.0,2019-05-10,0.441667,0.692424,0.23000,0.661,0.52600,0.9308,...,7.31,8.3,6.53,9,6th and 7th grade,79.09,6.6,8.3,6.53,381
1,Réveille Coffee,"I wanted to like this place, however the venti...",2.0,2019-04-20,0.046042,0.624306,0.14075,0.826,0.15800,0.1606,...,7.31,9.0,6.66,18,8th and 9th grade,68.10,8.7,9.0,6.66,745
2,Réveille Coffee,I didn't tried brunch before in another locati...,5.0,2019-04-18,0.500000,0.616667,0.00000,0.655,0.33950,0.9538,...,7.04,5.1,5.96,4,6th and 7th grade,79.97,4.2,5.1,5.96,189
3,Réveille Coffee,"Folks, avoid this place unless you like being ...",1.0,2019-04-16,0.073611,0.607778,0.22800,0.822,0.26225,0.6261,...,7.89,9.9,6.11,13,7th and 8th grade,68.50,8.6,9.9,6.11,763
4,Réveille Coffee,A nice compact coffee shop at Castro area. Qua...,4.0,2019-04-16,0.425000,0.610000,0.00000,0.647,0.35900,0.8689,...,8.77,6.6,7.59,5,6th and 7th grade,72.53,5.0,6.6,7.59,135


In [4]:
df_mining.drop(columns=['flesch', 'kincaid_grade', 'ari', 'dale_chall_readability'], inplace=True)

In [5]:
df_mining.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548 entries, 0 to 3547
Data columns (total 20 columns):
Coffee                              3548 non-null object
Description                         3548 non-null object
scores                              3548 non-null float64
valid_date                          3548 non-null object
polarity_textblob                   3548 non-null float64
subjectivity_textblob               3548 non-null float64
neg_nltk                            3548 non-null float64
neu_nltk                            3548 non-null float64
pos_nltk                            3548 non-null float64
compound_nltk                       3548 non-null float64
value_flair                         3548 non-null object
score_flair                         3548 non-null float64
coleman_liau_index                  3548 non-null float64
difficult_words                     3548 non-null int64
text_standard                       3548 non-null object
flesch_corrected              

In [6]:
def triScores(score):
    if score > 3:
        return 1
    elif score < 3:
        return -1
    else:
        return 0

In [7]:
df_mining['target'] = df_mining.scores.apply(triScores)

In [8]:
df_mining.groupby('target').count()

,Coffee,Description,scores,valid_date,polarity_textblob,subjectivity_textblob,neg_nltk,neu_nltk,pos_nltk,compound_nltk,value_flair,score_flair,coleman_liau_index,difficult_words,text_standard,flesch_corrected,kincaid_grade_corrected,ari_corrected,dale_chall_readability_corrected,len_review
target,,,,,,,,,,,,,,,,,,,,
-1,470,470,470,470,470,470,470,470,470,470,470,470,470,470,470,470,470,470,470,470
0,327,327,327,327,327,327,327,327,327,327,327,327,327,327,327,327,327,327,327,327
1,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751,2751


In [9]:
df_mining_sample = df_mining.groupby('target').apply(lambda x: x.sample(n=325, random_state=42)).reset_index(drop=True)

In [10]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df_mining_sample['code_coffee'] = lb_make.fit_transform(df_mining_sample["Coffee"])
df_mining_sample['code_text_standard'] = lb_make.fit_transform(df_mining_sample["text_standard"])
df_mining_sample['code_value_flair'] = lb_make.fit_transform(df_mining_sample["value_flair"])

In [11]:
y = df_mining_sample.target
X = df_mining_sample.drop(columns=['scores', 'target','Description', 'text_standard', 'Coffee', 'valid_date', 'value_flair'])
#X = df_mining_sample.drop(columns=['scores', 'target', 'Coffee','Description', 'valid_date', 'text_standard', 'value_flair'])

In [12]:
X.columns

Index(['polarity_textblob', 'subjectivity_textblob', 'neg_nltk', 'neu_nltk',
       'pos_nltk', 'compound_nltk', 'score_flair', 'coleman_liau_index',
       'difficult_words', 'flesch_corrected', 'kincaid_grade_corrected',
       'ari_corrected', 'dale_chall_readability_corrected', 'len_review',
       'code_coffee', 'code_text_standard', 'code_value_flair'],
      dtype='object')

In [13]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(f_classif, k=10)
fit = bestfeatures.fit(X, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat dataFrames
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(16,'Score'))  #print all features

                               Specs       Score
16                  code_value_flair  882.389430
5                      compound_nltk  223.367650
0                  polarity_textblob  171.801696
2                           neg_nltk  137.641978
3                           neu_nltk  103.331928
4                           pos_nltk   60.415035
6                        score_flair   25.165678
1              subjectivity_textblob   25.111055
13                        len_review   12.761526
10           kincaid_grade_corrected    6.180237
7                 coleman_liau_index    5.591446
8                    difficult_words    5.565207
11                     ari_corrected    3.667115
15                code_text_standard    1.988163
12  dale_chall_readability_corrected    1.427258
14                       code_coffee    1.387235


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(max_depth=8, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [15]:
y_pred = clf.predict(X_test)

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix(y_test, y_pred)

array([[67, 26,  3],
       [29, 62, 15],
       [ 1,  8, 82]])

In [17]:
accuracy_score(y_test, y_pred)

0.7201365187713311

In [18]:
from xgboost import XGBClassifier

In [19]:
#Inspecting performance adding estimators:
model = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 0.9,
                      objective='multi:softprob', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=3, 
                      gamma=1)

eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["merror"]
%time model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[0]	validation_0-merror:0.205279	validation_1-merror:0.296928
[1]	validation_0-merror:0.219941	validation_1-merror:0.296928
[2]	validation_0-merror:0.21261	validation_1-merror:0.293515
[3]	validation_0-merror:0.219941	validation_1-merror:0.293515
[4]	validation_0-merror:0.211144	validation_1-merror:0.293515
[5]	validation_0-merror:0.209677	validation_1-merror:0.286689
[6]	validation_0-merror:0.211144	validation_1-merror:0.296928
[7]	validation_0-merror:0.21261	validation_1-merror:0.296928
[8]	validation_0-merror:0.209677	validation_1-merror:0.286689
[9]	validation_0-merror:0.209677	validation_1-merror:0.286689
[10]	validation_0-merror:0.211144	validation_1-merror:0.283276
[11]	validation_0-merror:0.211144	validation_1-merror:0.293515
[12]	validation_0-merror:0.209677	validation_1-merror:0.286689
[13]	validation_0-merror:0.21261	validation_1-merror:0.293515
[14]	validation_0-merror:0.211144	validation_1-merror:0.286689
[15]	validation_0-merror:0.211144	validation_1-merror:0.283276
[16]	

[130]	validation_0-merror:0.203812	validation_1-merror:0.283276
[131]	validation_0-merror:0.205279	validation_1-merror:0.279863
[132]	validation_0-merror:0.205279	validation_1-merror:0.283276
[133]	validation_0-merror:0.205279	validation_1-merror:0.279863
[134]	validation_0-merror:0.205279	validation_1-merror:0.286689
[135]	validation_0-merror:0.206745	validation_1-merror:0.283276
[136]	validation_0-merror:0.208211	validation_1-merror:0.283276
[137]	validation_0-merror:0.206745	validation_1-merror:0.283276
[138]	validation_0-merror:0.206745	validation_1-merror:0.283276
[139]	validation_0-merror:0.206745	validation_1-merror:0.283276
[140]	validation_0-merror:0.205279	validation_1-merror:0.286689
[141]	validation_0-merror:0.205279	validation_1-merror:0.283276
[142]	validation_0-merror:0.205279	validation_1-merror:0.283276
[143]	validation_0-merror:0.205279	validation_1-merror:0.283276
[144]	validation_0-merror:0.205279	validation_1-merror:0.290102
[145]	validation_0-merror:0.206745	valid

[259]	validation_0-merror:0.184751	validation_1-merror:0.283276
[260]	validation_0-merror:0.184751	validation_1-merror:0.283276
[261]	validation_0-merror:0.183284	validation_1-merror:0.283276
[262]	validation_0-merror:0.184751	validation_1-merror:0.283276
[263]	validation_0-merror:0.186217	validation_1-merror:0.283276
[264]	validation_0-merror:0.186217	validation_1-merror:0.283276
[265]	validation_0-merror:0.186217	validation_1-merror:0.283276
[266]	validation_0-merror:0.183284	validation_1-merror:0.283276
[267]	validation_0-merror:0.184751	validation_1-merror:0.283276
[268]	validation_0-merror:0.184751	validation_1-merror:0.283276
[269]	validation_0-merror:0.184751	validation_1-merror:0.283276
[270]	validation_0-merror:0.183284	validation_1-merror:0.283276
[271]	validation_0-merror:0.184751	validation_1-merror:0.283276
[272]	validation_0-merror:0.183284	validation_1-merror:0.283276
[273]	validation_0-merror:0.183284	validation_1-merror:0.286689
[274]	validation_0-merror:0.183284	valid

[388]	validation_0-merror:0.164223	validation_1-merror:0.286689
[389]	validation_0-merror:0.162757	validation_1-merror:0.290102
[390]	validation_0-merror:0.162757	validation_1-merror:0.290102
[391]	validation_0-merror:0.162757	validation_1-merror:0.290102
[392]	validation_0-merror:0.162757	validation_1-merror:0.290102
[393]	validation_0-merror:0.162757	validation_1-merror:0.286689
[394]	validation_0-merror:0.162757	validation_1-merror:0.286689
[395]	validation_0-merror:0.162757	validation_1-merror:0.286689
[396]	validation_0-merror:0.16129	validation_1-merror:0.286689
[397]	validation_0-merror:0.16129	validation_1-merror:0.290102
[398]	validation_0-merror:0.16129	validation_1-merror:0.290102
[399]	validation_0-merror:0.158358	validation_1-merror:0.290102
[400]	validation_0-merror:0.158358	validation_1-merror:0.290102
[401]	validation_0-merror:0.16129	validation_1-merror:0.290102
[402]	validation_0-merror:0.158358	validation_1-merror:0.290102
[403]	validation_0-merror:0.158358	validatio

[517]	validation_0-merror:0.136364	validation_1-merror:0.290102
[518]	validation_0-merror:0.136364	validation_1-merror:0.290102
[519]	validation_0-merror:0.134897	validation_1-merror:0.290102
[520]	validation_0-merror:0.133431	validation_1-merror:0.290102
[521]	validation_0-merror:0.133431	validation_1-merror:0.290102
[522]	validation_0-merror:0.133431	validation_1-merror:0.290102
[523]	validation_0-merror:0.133431	validation_1-merror:0.290102
[524]	validation_0-merror:0.131965	validation_1-merror:0.290102
[525]	validation_0-merror:0.133431	validation_1-merror:0.290102
[526]	validation_0-merror:0.133431	validation_1-merror:0.290102
[527]	validation_0-merror:0.133431	validation_1-merror:0.290102
[528]	validation_0-merror:0.131965	validation_1-merror:0.290102
[529]	validation_0-merror:0.131965	validation_1-merror:0.290102
[530]	validation_0-merror:0.131965	validation_1-merror:0.290102
[531]	validation_0-merror:0.131965	validation_1-merror:0.290102
[532]	validation_0-merror:0.131965	valid

[646]	validation_0-merror:0.112903	validation_1-merror:0.307167
[647]	validation_0-merror:0.112903	validation_1-merror:0.307167
[648]	validation_0-merror:0.112903	validation_1-merror:0.307167
[649]	validation_0-merror:0.112903	validation_1-merror:0.307167
[650]	validation_0-merror:0.112903	validation_1-merror:0.307167
[651]	validation_0-merror:0.112903	validation_1-merror:0.307167
[652]	validation_0-merror:0.112903	validation_1-merror:0.307167
[653]	validation_0-merror:0.109971	validation_1-merror:0.307167
[654]	validation_0-merror:0.109971	validation_1-merror:0.307167
[655]	validation_0-merror:0.109971	validation_1-merror:0.307167
[656]	validation_0-merror:0.109971	validation_1-merror:0.31058
[657]	validation_0-merror:0.109971	validation_1-merror:0.307167
[658]	validation_0-merror:0.109971	validation_1-merror:0.307167
[659]	validation_0-merror:0.109971	validation_1-merror:0.307167
[660]	validation_0-merror:0.109971	validation_1-merror:0.307167
[661]	validation_0-merror:0.109971	valida

[776]	validation_0-merror:0.092375	validation_1-merror:0.313993
[777]	validation_0-merror:0.092375	validation_1-merror:0.313993
[778]	validation_0-merror:0.092375	validation_1-merror:0.313993
[779]	validation_0-merror:0.092375	validation_1-merror:0.313993
[780]	validation_0-merror:0.092375	validation_1-merror:0.313993
[781]	validation_0-merror:0.093842	validation_1-merror:0.313993
[782]	validation_0-merror:0.093842	validation_1-merror:0.313993
[783]	validation_0-merror:0.092375	validation_1-merror:0.313993
[784]	validation_0-merror:0.092375	validation_1-merror:0.313993
[785]	validation_0-merror:0.092375	validation_1-merror:0.313993
[786]	validation_0-merror:0.092375	validation_1-merror:0.313993
[787]	validation_0-merror:0.092375	validation_1-merror:0.313993
[788]	validation_0-merror:0.092375	validation_1-merror:0.313993
[789]	validation_0-merror:0.093842	validation_1-merror:0.313993
[790]	validation_0-merror:0.092375	validation_1-merror:0.313993
[791]	validation_0-merror:0.092375	valid

[905]	validation_0-merror:0.08651	validation_1-merror:0.317406
[906]	validation_0-merror:0.08651	validation_1-merror:0.317406
[907]	validation_0-merror:0.08651	validation_1-merror:0.317406
[908]	validation_0-merror:0.08651	validation_1-merror:0.317406
[909]	validation_0-merror:0.08651	validation_1-merror:0.317406
[910]	validation_0-merror:0.08651	validation_1-merror:0.317406
[911]	validation_0-merror:0.08651	validation_1-merror:0.317406
[912]	validation_0-merror:0.08651	validation_1-merror:0.317406
[913]	validation_0-merror:0.08651	validation_1-merror:0.317406
[914]	validation_0-merror:0.08651	validation_1-merror:0.317406
[915]	validation_0-merror:0.08651	validation_1-merror:0.317406
[916]	validation_0-merror:0.08651	validation_1-merror:0.317406
[917]	validation_0-merror:0.08651	validation_1-merror:0.317406
[918]	validation_0-merror:0.08651	validation_1-merror:0.317406
[919]	validation_0-merror:0.08651	validation_1-merror:0.317406
[920]	validation_0-merror:0.08651	validation_1-merror:0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.9, verbosity=1)

In [20]:
xg = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 0.9,
                      objective='multi:softprob', 
                      n_estimators=260, 
                      reg_alpha = 0.3,
                      max_depth=3, 
                      gamma=1)

In [21]:
xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=260, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.9, verbosity=1)

In [22]:
y_pred = xg.predict(X_test)
predictions = [round(value) for value in y_pred]

In [23]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.67%


In [24]:
### Over-Sampling

In [26]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy={-1:1000, 0:1000, 1:1000}, random_state=42)
X_res, y_res = ros.fit_resample(X, y)

Using TensorFlow backend.


In [27]:
X_trainres, X_testres, y_trainres, y_testres = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

In [28]:
#Inspecting performance adding estimators:
model = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 0.9,
                      objective='multi:softprob', 
                      n_estimators=200, 
                      reg_alpha = 0.3,
                      max_depth=5, 
                      gamma=1)

eval_set = [(X_trainres, y_trainres), (X_testres, y_testres)]
eval_metric = ["merror"]
%time model.fit(X_trainres, y_trainres, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

[0]	validation_0-merror:0.170476	validation_1-merror:0.22
[1]	validation_0-merror:0.156667	validation_1-merror:0.227778
[2]	validation_0-merror:0.152381	validation_1-merror:0.222222
[3]	validation_0-merror:0.147143	validation_1-merror:0.213333
[4]	validation_0-merror:0.148095	validation_1-merror:0.21
[5]	validation_0-merror:0.148095	validation_1-merror:0.203333
[6]	validation_0-merror:0.150952	validation_1-merror:0.204444
[7]	validation_0-merror:0.14381	validation_1-merror:0.212222
[8]	validation_0-merror:0.132381	validation_1-merror:0.202222
[9]	validation_0-merror:0.133333	validation_1-merror:0.195556
[10]	validation_0-merror:0.133333	validation_1-merror:0.194444
[11]	validation_0-merror:0.131905	validation_1-merror:0.196667
[12]	validation_0-merror:0.139524	validation_1-merror:0.2
[13]	validation_0-merror:0.137143	validation_1-merror:0.197778
[14]	validation_0-merror:0.139524	validation_1-merror:0.196667
[15]	validation_0-merror:0.134286	validation_1-merror:0.195556
[16]	validation_

[132]	validation_0-merror:0.105238	validation_1-merror:0.168889
[133]	validation_0-merror:0.105238	validation_1-merror:0.168889
[134]	validation_0-merror:0.101905	validation_1-merror:0.167778
[135]	validation_0-merror:0.101905	validation_1-merror:0.167778
[136]	validation_0-merror:0.101905	validation_1-merror:0.167778
[137]	validation_0-merror:0.100952	validation_1-merror:0.166667
[138]	validation_0-merror:0.100952	validation_1-merror:0.166667
[139]	validation_0-merror:0.099048	validation_1-merror:0.166667
[140]	validation_0-merror:0.099048	validation_1-merror:0.166667
[141]	validation_0-merror:0.099048	validation_1-merror:0.166667
[142]	validation_0-merror:0.099048	validation_1-merror:0.166667
[143]	validation_0-merror:0.099048	validation_1-merror:0.166667
[144]	validation_0-merror:0.099048	validation_1-merror:0.166667
[145]	validation_0-merror:0.099048	validation_1-merror:0.166667
[146]	validation_0-merror:0.097619	validation_1-merror:0.166667
[147]	validation_0-merror:0.09619	valida

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.9, verbosity=1)

In [29]:
xgb_model = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 1,
                      subsample = 0.9,
                      objective='multi:softprob', 
                      n_estimators=180, 
                      reg_alpha = 0.3,
                      max_depth=5, 
                      gamma=1)

xg.fit(X_trainres, y_trainres)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=260, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.9, verbosity=1)

In [30]:
y_pred = xg.predict(X_testres)
predictions = [round(value) for value in y_pred]

In [31]:
accuracy = accuracy_score(y_testres, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 76.78%
